In [1]:
import sys
sys.path.insert(1, '../../..')

In [2]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.preprocessing import MultiLabelBinarizer
from medicine.models.compound import Compound
from mlxtend.preprocessing import TransactionEncoder

In [3]:
compound_list = []
for compound in Compound.query.all():
    medicine_list = [medicine.name for medicine in compound.medicine]
    compound_list.append(medicine_list)

F:\Python\anaconda3\lib\site-packages\sqlalchemy\engine\default.py:470: Warning: (1366, "Incorrect string value: '\\xA5x\\xA5_\\xBC\\xD0...' for column 'VARIABLE_VALUE' at row 480")
  cursor.execute(statement, parameters)


In [4]:
df = pd.DataFrame({'medicines':compound_list})
df.head(5)

,medicines
0,[夏冰]
1,[半天河]
2,[半天河]
3,"[流水, 半夏]"
4,"[流水, 甘草, 茯苓]"


In [5]:
# te = TransactionEncoder()
# te_ary = te.fit(compound_list).transform(compound_list)
# df = pd.DataFrame(te_ary, columns=te.columns_)
# df.head(10)

In [6]:
mlb = MultiLabelBinarizer()
basket = pd.DataFrame(mlb.fit_transform(df['medicines']),
                      columns=mlb.classes_,
                      index=df.index)

In [7]:
basket.head(10)

,丁香,三七,不灰木,丹參,丹砂,丹黍米,乳汁,乳腐,乾薑,亂發,...,鼠李,鼬鼠,齑水,齒,龍珠,龍眼,龍腦香,龍膽,龍舌草,龍葵
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
frequent_itemsets = apriori(basket, min_support=0.0005, use_colnames=True, max_len=2)
frequent_itemsets.head(5)

,support,itemsets
0,0.003983,[丁香]
1,0.003802,[三七]
2,0.001086,[不灰木]
3,0.001086,[丹參]
4,0.019917,[丹砂]


In [9]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head(5)

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(人參),(丁香),0.019374,0.003983,0.000724,0.037383,9.384877,0.000647,1.034697
1,(丁香),(人參),0.003983,0.019374,0.000724,0.181818,9.384877,0.000647,1.198543
2,(丁香),(附子),0.003983,0.007424,0.000543,0.136364,18.369180,0.000514,1.149299
3,(附子),(丁香),0.007424,0.003983,0.000543,0.073171,18.369180,0.000514,1.074650
4,(不灰木),(玄精石),0.001086,0.002173,0.000724,0.666667,306.833333,0.000722,2.993482


In [10]:
datas = rules.head(5).to_json(lines=False, orient='records', force_ascii=False)
datas

'[{"antecedants":["人參"],"consequents":["丁香"],"antecedent support":0.0193735289,"consequent support":0.0039833424,"support":0.0007242441,"confidence":0.0373831776,"lift":9.3848768054,"leverage":0.0006470727,"conviction":1.0346969162},{"antecedants":["丁香"],"consequents":["人參"],"antecedent support":0.0039833424,"consequent support":0.0193735289,"support":0.0007242441,"confidence":0.1818181818,"lift":9.3848768054,"leverage":0.0006470727,"conviction":1.1985434647},{"antecedants":["丁香"],"consequents":["附子"],"antecedent support":0.0039833424,"consequent support":0.0074235017,"support":0.0005431831,"confidence":0.1363636364,"lift":18.3691796009,"leverage":0.0005136127,"conviction":1.1492991033},{"antecedants":["附子"],"consequents":["丁香"],"antecedent support":0.0074235017,"consequent support":0.0039833424,"support":0.0005431831,"confidence":0.0731707317,"lift":18.3691796009,"leverage":0.0005136127,"conviction":1.0746495516},{"antecedants":["不灰木"],"consequents":["玄精石"],"antecedent support":0.0010

In [11]:
import json

In [12]:
# data_str = json.dumps(datas)
j = json.loads(datas)

In [13]:
{'data':j}

{'data': [{'antecedants': ['人參'],
   'antecedent support': 0.0193735289,
   'confidence': 0.0373831776,
   'consequent support': 0.0039833424,
   'consequents': ['丁香'],
   'conviction': 1.0346969162,
   'leverage': 0.0006470727,
   'lift': 9.3848768054,
   'support': 0.0007242441},
  {'antecedants': ['丁香'],
   'antecedent support': 0.0039833424,
   'confidence': 0.1818181818,
   'consequent support': 0.0193735289,
   'consequents': ['人參'],
   'conviction': 1.1985434647,
   'leverage': 0.0006470727,
   'lift': 9.3848768054,
   'support': 0.0007242441},
  {'antecedants': ['丁香'],
   'antecedent support': 0.0039833424,
   'confidence': 0.1363636364,
   'consequent support': 0.0074235017,
   'consequents': ['附子'],
   'conviction': 1.1492991033,
   'leverage': 0.0005136127,
   'lift': 18.3691796009,
   'support': 0.0005431831},
  {'antecedants': ['附子'],
   'antecedent support': 0.0074235017,
   'confidence': 0.0731707317,
   'consequent support': 0.0039833424,
   'consequents': ['丁香'],
   'c

In [14]:
rules.head(2).to_json(lines=True, orient='records', force_ascii=False)

'{"antecedants":["人參"],"consequents":["丁香"],"antecedent support":0.0193735289,"consequent support":0.0039833424,"support":0.0007242441,"confidence":0.0373831776,"lift":9.3848768054,"leverage":0.0006470727,"conviction":1.0346969162}\n{"antecedants":["丁香"],"consequents":["人參"],"antecedent support":0.0039833424,"consequent support":0.0193735289,"support":0.0007242441,"confidence":0.1818181818,"lift":9.3848768054,"leverage":0.0006470727,"conviction":1.1985434647}'